### Objective

Get unique IDs from a flat file.

### Restrictions

1. The program can only run in a single machine, not a distribution system

2. The flat file is too big to fit in the memory

3. The unique IDs are also too big to fit in the memory.

### Assumptions

- Characters in these IDs: only alpha and numeric character a-z|A-Z|0-9, which is 62 chars in total. The reason is many uniq ID generating system will use these common chars to produced an ID.

- Length of a single ID: It doesn’t matter much as the length of IDs are not vary too much, and each ID could fix in the memory. :) Let’s assume the length would be 10 chars for simplicity.

  With the two assumptions, the total number of uniqe ID is $62^7 = 3,521,614,606,208$. Let’s say 1% of these numbers are in the final unique ID list.  It will take 246GB  ($62^7 / 100 * 7B$) space, which cannot fit in most computer’s RAM.

- The original file is around 50GB (~7 billioin lines),  and my computer has 8GB RAM memory and 1TB disk space.

In [1]:
### Prepare a input ID file for demostration purpose: input_id.txt

import random

def gen_ids(valid_chars, length, N):
    """Generate N number of id:
    - each char of the id is in valid_chars
    - the lenght of id is length
    """
    ids = []
    for n in range(N):
        id = random.choices(valid_chars, k=length)
        ids.append(''.join(id))
    return ids

In [2]:
alpha = 'abcdefghigklmnopqrstuvwxyz'
nums = '01234567890'
CHARS = alpha + alpha.upper() + nums

file_name = 'input_id.txt' # file for generated IDs
length = 7
N = 10000 # 10 kilo, could be 7 billion :)

with open(file_name, 'w') as f:
    f.write('\n'.join(gen_ids(CHARS, length, N)))

In [3]:
gen_ids(CHARS, 7, 10)

['2PlyIKp',
 'PD00Lb4',
 'enQ1kYy',
 'bVGIGTn',
 'b1XmYDq',
 'KZeXgWC',
 'vZvc2yT',
 'Sh28z05',
 'yC0Lklb',
 'SAr7D4w']

### Solution 1: Count the ID starting with a, b, c...

Count the number of unique IDs start each char of a-zA-Z0-9, and then append them in a file. Thus, this file contains all the unique IDs of the input file. 

This approach assumes all IDs starting with any char could fit in the memory as there is no hotspot. We can do this by sampling the input file (1 million lines, 7MB size) and check the distribution.

In [4]:
# Here is the command in **Linux** (not working in macOS) to get the ramdon n lines from input file and save it to the sample file.

!shuf -n 1000000 input_id.txt > sample_id.txt

# Note: This command doesn't work in macOS.

/bin/sh: shuf: command not found


In [23]:
# check the distribution of the first char (a-zA-Z0-9) of input IDs

import fileinput
from collections import defaultdict

counter = defaultdict(int)
with fileinput.input(files='sample_id.txt') as f:    
    for i in f:
        counter[i[:1]] += 1

for key, value in counter.items():
    print(key, value)

w 176
I 163
N 172
r 157
H 175
c 158
2 150
4 184
E 161
0 316
D 161
F 159
f 139
u 158
W 167
y 163
S 160
h 170
3 161
A 157
B 157
q 146
g 307
l 174
Y 134
T 168
6 170
p 151
x 157
k 147
d 157
M 150
G 296
8 170
O 133
9 178
Z 168
s 144
V 178
e 164
b 145
5 161
i 147
n 144
R 171
X 158
o 148
U 158
z 171
t 164
v 170
1 156
7 145
K 180
a 151
m 167
C 145
Q 134
L 170
P 159


In [24]:
# create a new folder for splited id files
! mkdir splits

# split the big file to small pieces (70 million lines, ~500 MB) using linux command: split
! split -l 70000000 input_id.txt splits/

In [25]:
import os
import fileinput
import collections

# uniq_file = 'unique_id.txt'

#Return a set of IDs started with `start_char` in file `file`
def extract_uniq_ids(file, start_char):
    uniq_ids = set() # id start with 'start_char'
    with fileinput.input(files = file) as lines:
        for line in lines:
            if line.startswith(start_char) and line not in uniq_ids:
                uniq_ids.add(line.strip())
    return uniq_ids

In [26]:
prefix = 'id-'

for root, dirs, files in os.walk("splits"):
    for start_char in CHARS:
        uniq_ids = set()
        for file_name in files:
            if file_name.startswith('.'):
                continue # bugfix: skip the hidden files from OS
            
            file = os.sep.join((root, file_name))
            uniq_ids = uniq_ids.union(extract_uniq_ids(file, start_char))
        
        if start_char.isupper(): # bugfix for case insensitive OS like mac
            start_char += '_'
            
        if uniq_ids:
            with open(prefix + start_char + '.txt', 'w') as f:
                f.write('\n'.join(uniq_ids))
                f.write('\n')

In [27]:
# simple merge these small files into a big one, the number should be equal the uniq id in the orignal file.

!cat id-*.txt > uniq_id.txt
!wc -l uniq_id.txt

   10000 uniq_id.txt


In [28]:
# count the uniq numer of id using linux command to verify.

!sort uniq_id.txt | uniq | wc -l

   10000


The number matches. 10000 records is too small to get duplicated ID. Try 7 billion records will be anohter history.

#### Unit Test

Clean the folder, and put this file as 'unit_test.txt' in the splits folder and run the above programs.
```
xNDGN3R
8guP0Af
VHgwqgA
Wy0AXoo
wy0AXoo
xNDGN3R
toS3f6T
8bIgIXy
xNDGN3R
```

In [29]:
# Then use linux command cat, sort, uniq to merge these files and count the number of uniq files.
!sort splits/unit_test.txt | uniq | wc -l

sort: No such file or directory
       0


**So the final file will be 'uniq_id.txt'.**

#### Optimization

The bottleneck of this program is the multiple I/O read, becuase it will read the splitted files 62 times as each start char counts once.

To reduce the heavy I/O, I could:
1. Read one split file each time.
2. Get all the IDs in a map, the key of which is the start char such as a, b, c... 7, 8, 9, and the value of which is a set of IDs
3. For each <key, value> in step 2, update the coresponding output file by merge the IDs to the existing ones by:
   Read the existing/new smaill id-x file in a set;
   Merge the set with the value set in the map of step 2
   Flush the merged set to the disk

Here is the implementation in Python.

In [31]:
prefix = 'id-'
CHARS_SET = set(CHARS) # for quick lookup

import fileinput

from collections import defaultdict

def extract_ids(file):
    uniq_ids = defaultdict(set) # k:'[a-z|A-Z|0-9]', value = set()
    with fileinput.input(files=file) as f:
        for line in f:
            first_char = line[:1]
            if first_char in CHARS_SET:
                uniq_ids[first_char].add(line.strip())
    return uniq_ids

In [32]:
extract_ids('splits/unit_test.txt')

defaultdict(set,
            {'x': {'xNDGN3R'},
             '8': {'8bIgIXy', '8guP0Af'},
             'V': {'VHgwqgA'},
             'W': {'Wy0AXoo'},
             'w': {'wy0AXoo'},
             't': {'toS3f6T'}})

In [33]:
import os.path

def merge_ids(file, other_ids):
    ids = set()
    
    if os.path.isfile(file): 
        with open(file, 'r') as f:
            ids = set(line.strip() for line in f.readlines())
            f.close()
    
    ids = ids.union(set(other_ids))
    
    with open(file, 'w') as f:
        f.write('\n'.join(ids))
        f.write('\n')
    return ids

In [34]:
for root, dirs, files in os.walk("splits"):
    for file_name in files:
        if file_name.startswith('.'):
            continue # bugfix: skip the hidden files from OS

        file = os.sep.join((root, file_name))
        for start_char, id_set in extract_ids(file).items():
            if start_char.isupper(): # bugfix for case insensitive OS like mac
                start_char += '_'
            file_to_merge = prefix + start_char + '.txt'
            merge_ids(file_to_merge, id_set)

#### Limitation
If the IDs are highly skewed (huge ID starts with the same char), then this method won't work because those group of IDs cannot fix in memory.

### Solution 2: Use database column's unique feature

**Intution**
1. Take the same step as before to split the files into small files.
2. Create a table `unique_id` with one column `id` which is unique in MySQL.
3. Take one files and read all the IDs, then batch insert these values into the database using `insert ignore`.
4. Continue to process all the splitted files till the end.

The reason why it works is because the duplicated values are ignored. Here is the SQL code for reference.

```sql
drop table unique_id;

create table unique_id (id char(7) UNIQUE not NULL);

insert ignore into unique_id VALUES
(‘xNDGN3R),
(‘8guP0Af),
(‘VHgwqgA),
(‘Wy0AXoo),
(‘wy0AXoo),
(‘xNDGN3R),
(‘toS3f6T),
(‘8bIgIXy),
(‘xNDGN3R);
```

**Verification**
select * from unique_id;

```
id
---
8bIgIXy
8guP0Af
toS3f6T
VHgwqgA
Wy0AXoo
xNDGN3R
```